In [5]:
# pip install -r requirements.txt

Processing /home/conda/feedstock_root/build_artifacts/autovizwidget_1652515004592/work
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/conda/feedstock_root/build_artifacts/autovizwidget_1652515004592/work'

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import ray, os
ray.init(num_cpus=96)
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as md

In [12]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from dotenv import load_dotenv
load_dotenv('env')
from importlib import reload
from modules import DataCollection
reload(DataCollection)

s3_client = DataCollection.s3_client()

In [13]:
if not os.path.exists("raw_data"):
    os.mkdir("raw_data")
    
    for file_name in ['business.csv']:
        s3_client.download_file('s3fld','599Team12/raw_data/%s'% file_name,'raw_data/%s'% file_name) 

In [14]:
business = md.read_csv('raw_data/business.csv')
review = md.read_csv('raw_data/review.csv')

2022-11-16 02:30:29,266	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.


In [15]:
resturant_business = business[business['categories'].str.lower().str.contains('restaurants', na=False)]
resturant_business = resturant_business[['business_id','name','address','city','state','postal_code','latitude','longitude','stars']]
resturant_business.sample(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars
122977,e2_MIdH-CzuWSmLQ7vNC9A,Green Papaya Asian Bistro,202 E Lancaster Ave,Ardmore,PA,19003,40.004917,-75.285750,4.5
87305,SlfJo2ebEtNxcPGlcOx79A,Spencer Eta Burger,227 N 34th St,Philadelphia,PA,19104,39.959154,-75.190834,4.0
17543,xbCmAKZ0AzA2LsF-HWyxIQ,Mickey Quinns Crafthouse and Eatery,9057 Ulmerton Rd,Largo,FL,33771,27.894567,-82.764570,4.5
21366,m6RGfATGF1aL_g7oYJ-mbg,honeygrow,3731 Walnut St,Philadelphia,PA,19104,39.953732,-75.197820,3.0
124308,Hzw_macg-BQ1iwgdxwteHw,Julie Anne's Place,22 S Warren Ave,Malvern,PA,19355,40.035359,-75.514413,4.5


In [20]:
resturant_review = review[review['business_id'].isin(resturant_business['business_id'])]
resturant_review['date'] = pd.to_datetime(resturant_review['date'], format='%Y-%m-%d')
resturant_review = resturant_review[['review_id', 'business_id','stars','text','date']]
resturant_review.sample(5)

(apply_list_of_funcs pid=31418) 2022-11-16 02:34:00,531	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=31409) 2022-11-16 02:34:00,550	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=24376) 2022-11-16 02:34:00,551	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=24433) 2022-11-16 02:34:00,519	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=24213) 2022-11-16 02:34:00,533	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed 

,review_id,business_id,stars,text,date
2143635,kaDxzzH8JodMpsD-4xpaOw,Rl1NN7QH08h5h0XO00lPFA,5,Rubicon Deli is great! Super small sandwich sh...,2016-03-07 23:10:40
6449944,oxuPGy5AeXpBmWypABRwfg,ws0mjFrkanX_sXoUVRh8Uw,4,Baristas who know what they're doing are a ref...,2011-10-18 17:52:06
6411654,a__6TqDzq5pWDl8J9g9ixg,MLZMNL8R6zTJgPDB7SUQnA,5,This place is the best! There's nothing fancy...,2013-03-26 00:03:50
4700707,iRFZLBmssHiFGcsnEIRX0g,offiWjhY-INp-m6dEGi80g,5,We love this place! Angelina is suuuuuuch a sw...,2020-02-16 09:34:18
1963254,0Nrus89w5CF1BMrkDm-tKA,oH3noIG2wC7DbDUChD9r7g,5,First time visiting Maryland Crab Seafood Hous...,2020-07-12 16:12:50


In [12]:
resturant_name_dict = resturant_business.set_index('business_id')['name'].to_dict()
resturant_review['resturant_name'] = resturant_review['business_id'].map(resturant_name_dict)
resturant_review.sample(5)

,review_id,business_id,stars,text,date,resturant_name
926241,dzZy0zUfzwUYEJQjsmshcg,z3TB52s5p5PcRsw2T3J6MQ,2,This place is over rated and charged for every...,2018-06-15 01:54:06,Chicken and Taco Loco
1690545,RSjw8ETHKxow8YWLcP6FFQ,U3grYFIeu6RgAAQgdriHww,5,"This place was phenomenal. Coming from Boston,...",2015-10-03 03:29:40,Brophy Bros - Santa Barbara
6420406,sGpD0wCJMJV7MTmAzpLYrA,3CGXa85uNgvp8Berx5AhQQ,4,The main courses are excellent. I've tried th...,2014-09-14 22:29:03,La Margarita
5086014,RBh8gjvFulFiInbV7ya8YA,cQIh4YJlVtZI9TLF5_smOg,4,"So, I arrived a little tipsy at the Shaved Duc...",2011-02-23 23:35:33,The Shaved Duck
5241203,juhAHcBE-nDnft5REaY17Q,OxQK9EMUXqynT7dbWAP9wA,5,"Fantastic BBQ joint!! Great atmosphere, tende...",2019-05-26 20:25:01,Mission BBQ


In [21]:
resturant_review['sentiment'] = resturant_review['stars'].apply(lambda x: 'negative' if x <= 2 else ('positive' if x >= 4 else 'neutral'))
resturant_review.sample(5)

,review_id,business_id,stars,text,date,sentiment
2134244,Qai4k95Me0Rmhth3P4riYA,B4JZy1PBgiO3xp8zKgwUJA,3,So I've heard nothing but good things about Da...,2018-02-01 22:07:37,neutral
4156348,VwA6ZeT8npZZNUq0Q76NlA,XPO4VKjYa-W7SSvQqFRubg,5,We finally went back for dinner after visiting...,2019-06-16 23:14:58,positive
4701867,IujO74WBCIuY-HhqSKrkrQ,cOihnyvkuI4M6wLEzHcTzg,4,"Mr Bubbles, you are the banh for mi! The banh ...",2021-07-26 13:18:46,positive
1698348,yZF63YF2l3UoLwFhbUDrow,K6SQfxBapNAIVr_C73ohlw,5,Love ordering here for lunch! Their Thai green...,2016-01-13 05:35:14,positive
3695863,QZLyoF-R_Hg6S8kIv1EZTA,c3gwn7_ukRwGGIghHLi-UA,5,"Amazing! Love the food, service, and atmospher...",2018-01-28 20:54:19,positive


In [22]:
if not os.path.exists("restaurants"):
    os.mkdir("restaurants")
    
resturant_business.to_csv('restaurants/business.csv', compression='zip')
resturant_review.to_csv('restaurants/review.csv', compression='zip')

In [27]:
# this step only works if authorized user have the enviroment variables

s3_client.upload_file('restaurants/business.csv', 's3fld', '599Team12/restaurants/business.csv', ExtraArgs={'ACL': 'public-read'})
s3_client.upload_file('restaurants/review.csv', 's3fld', '599Team12/restaurants/review.csv', ExtraArgs={'ACL': 'public-read'})